In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [ ]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")

INFO:tensorflow:Using /var/folders/xl/xw9cp_fd22qbyf3ff_008yh80000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.


In [5]:
sess = tf.Session()
sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
messages = tf.placeholder(dtype=tf.string, shape=[None])
embeddings = embed(messages)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [6]:
# Read terms and write embedding data to file.

import csv
data_file = "cleaned_data.tsv"
write_to = "embedding_data.tsv"
terms_with_embeddings = []

with open(data_file, 'r') as f:
    file = csv.reader(f, delimiter='\t')
    for line in file:
        to_embed = line[0]
        parent = line[1]
        term_vector = sess.run(embeddings, feed_dict={messages: [to_embed]})
        terms_with_embeddings.append([to_embed, parent, term_vector])
    f.close()

with open(write_to, 'w') as f:
    for triple in terms_with_embeddings:
        f.write('%s\t%s\t%s\n' % (triple[0], triple[1], triple[2]))
    f.close()

In [51]:
from scipy import spatial
import heapq

def logistic(z):
    """ Modified logistic exponential to score cosine similarity strength. """
    return 1.0495 / (1.0000 + (3.0000 * np.exp((-4.1000) * z)))

def cosine_similarity(matrix1, matrix2):
    """ Calculates cosine similarity between two vectors. """
    return 1 - spatial.distance.cosine(matrix1, matrix2)

def top_similar(feature):
    term_map = {}
    my_matrix = sess.run(embeddings, feed_dict={messages: [feature]})
    
    for i, term in enumerate(terms_with_embeddings):
        term_map[term[0]] = cosine_similarity(my_matrix, term[2])
        
    bestResult = heapq.nlargest(1, term_map, key=term_map.get)
    return_val = bestResult[0].split(', ')
    
    return (return_val[0], return_val[1], str(round(logistic(term_map[bestResult[0]]), 2) * 100) + "%")

In [62]:
print(top_similar("gastrointestinal"))
print(top_similar("VC firms invested in this company"))
print(top_similar("a new approach"))
print(top_similar("marketing aspects"))

('Digestive system', ' intestinal pseudoobstruction', '96.0%')
('Financial Features', 'number of venture capital firms investing in the company', '93.0%')
('Innovation Model', 'novel approach', '83.0%')
('Managerial Features', 'marketing side', '90.0%')
